In the Summer of 2023 HERO fellows collected data on trees planted by the MA DCR in Worcester Massachusetts
This data came to reside in "LBSurvey23FINAL.csv"
I perfomed preliminary modifications on this resulting in "S23_work.csv"

In the Fall of 2023 a smaller HERO contingent collected data on trees from the same DCR planting program throughout a wider swath of the longhorned beetle quarantine/replanting zone, including some or all of towns surrounding Worcester
This data ended up in "Dataset_for_final_analysis_fall23.csv"

All trees in question were planted between Fall 2010, and Spring 2012 as part of the effort to replant after the longhorned beetle removals.
A baseline survey of most of the trees surveyed in 2023 was conducted in 2014, 2015, and 2015. 
This baseline data is found in "DCRreplanted141516gisfieldmap.csv"

This notebook is a first step towards unifying the two 2023 survey datasheets to be merged.
Most of the modifications made here involve conforming the fields of S23 to the field structure of F23. 
Additionally, coordinate data for S23 trees is added via a join with baseline

In [1]:
import numpy as np
import geopandas as gpd
import pandas as pd

In [2]:
# This was done prior to current iteration of .csv files
"""
# Assuming S23_work is your DataFrame
# Convert the date columns to datetime format
S23_work['Observatio'] = pd.to_datetime(S23_work['Observatio'], errors='coerce')
S23_work['Observdate'] = pd.to_datetime(S23_work['Observdate'], errors='coerce')
S23_work['Observat_1'] = pd.to_datetime(S23_work['Observat_1'], errors='coerce')

# Create a new column "Observation_Date" with the most recent date
S23_work['Observation_Date'] = S23_work[['Observatio', 'Observdate', 'Observat_1']].max(axis=1)

# Drop the original date columns if needed
S23_work.drop(['Observatio', 'Observdate', 'Observat_1'], axis=1, inplace=True)

# Print the updated DataFrame
print(S23_work)
"""

'\n# Assuming S23_work is your DataFrame\n# Convert the date columns to datetime format\nS23_work[\'Observatio\'] = pd.to_datetime(S23_work[\'Observatio\'], errors=\'coerce\')\nS23_work[\'Observdate\'] = pd.to_datetime(S23_work[\'Observdate\'], errors=\'coerce\')\nS23_work[\'Observat_1\'] = pd.to_datetime(S23_work[\'Observat_1\'], errors=\'coerce\')\n\n# Create a new column "Observation_Date" with the most recent date\nS23_work[\'Observation_Date\'] = S23_work[[\'Observatio\', \'Observdate\', \'Observat_1\']].max(axis=1)\n\n# Drop the original date columns if needed\nS23_work.drop([\'Observatio\', \'Observdate\', \'Observat_1\'], axis=1, inplace=True)\n\n# Print the updated DataFrame\nprint(S23_work)\n'

In [3]:
F23 = gpd.read_file("Input_Data/Dataset_for_final_analysis_fall23.csv")
baseline = gpd.read_file("Input_Data/DCRreplanted141516gisfieldmap.csv")
S23 = gpd.read_file("Input_Data/S23_work.csv")

In [4]:
# Set display options to show all columns
pd.set_option('display.max_columns', None)

print(S23.head())

  OBJECTID   ID OID_          SPECIES        STREET HOUSE_NUMBER       TOWN  \
0        1  956  110   Sargent Cherry  ACUSHNET AVE            9  Worcester   
1        2  962   34     Dawn Redwood     AIRLIE ST           70  Worcester   
2        3  963   48     Serviceberry     AIRLIE ST           70  Worcester   
3        4  964   35  Swamp White Oak     AIRLIE ST           70  Worcester   
4        5  965   38         Blackgum     AIRLIE ST           74  Worcester   

  SEASON_PLANTED Observation_Date_baseline SiteType_baseline LandUse_baseline  \
0      Fall 2010                1899-12-30           Unknown          Unknown   
1      Fall 2010                 6/23/2015         Back Yard    Single-family   
2      Fall 2010                 6/23/2015        Front yard    Single-family   
3      Fall 2010                 6/23/2015         Back Yard    Single-family   
4      Fall 2010                 6/23/2015         Back Yard    Single-family   

  DBH_baseline DBH_Height_baseline DBH

In [5]:
# Preliminary data cleaning
# Replace entries containing "1899" with "NA" in 'Observation_Date_baseline' column
S23.loc[S23['Observation_Date_baseline'].astype(str).str.contains("1899"), 'Observation_Date_baseline'] = "NA"
# Replace 'Unknown' with NA in 'SiteType_baseline' and 'LandUse_baseline' columns
S23.loc[S23['SiteType_baseline'].astype(str).str.contains("Unknown"), 'SiteType_baseline'] = "NA"
S23.loc[S23['LandUse_baseline'].astype(str).str.contains("Unknown"), 'LandUse_baseline'] = "NA"

# Replace values in 'Vigor_baseline' column
S23['Vigor_baseline'].replace({
    '1-25%': '1',
    '26-50%': '2',
    '51-75%': '3',
    '76-100%': '4'
}, inplace=True)

# Concatenate values from 'Comments_1', 'Comments_2', and 'Comments_3' into 'Comments_baseline'
S23['Comments_baseline'] = S23['Comments_1'].fillna('') + ' ' + S23['Comments_2'].fillna('') + ' ' + S23['Comments_3'].fillna('')

# Drop the original 'Comments_1', 'Comments_2', and 'Comments_3' columns if needed
S23.drop(['Comments_1', 'Comments_2', 'Comments_3'], axis=1, inplace=True)

# Capitalize all letters and remove commas in 'Crew_Initials' column
S23['Crew_Initials'] = S23['Crew_Initials'].str.upper().str.replace(',', '')

In [6]:
# Perform a left join on 'ID' column to add 'LONGITUDE' and 'LATITUDE' from baseline to S23
S23 = pd.merge(S23, baseline[['ID', 'LONGITUDE', 'LATITUDE']], on='ID', how='left')

In [7]:
# Check the number of rows in S23
# Should still be 2381
S23_rows = len(S23)
print("Number of rows:", S23_rows)

# Double check everything looks right
print(S23.head())

Number of rows: 2381
  OBJECTID   ID OID_          SPECIES        STREET HOUSE_NUMBER       TOWN  \
0        1  956  110   Sargent Cherry  ACUSHNET AVE            9  Worcester   
1        2  962   34     Dawn Redwood     AIRLIE ST           70  Worcester   
2        3  963   48     Serviceberry     AIRLIE ST           70  Worcester   
3        4  964   35  Swamp White Oak     AIRLIE ST           70  Worcester   
4        5  965   38         Blackgum     AIRLIE ST           74  Worcester   

  SEASON_PLANTED Observation_Date_baseline SiteType_baseline LandUse_baseline  \
0      Fall 2010                        NA                NA               NA   
1      Fall 2010                 6/23/2015         Back Yard    Single-family   
2      Fall 2010                 6/23/2015        Front yard    Single-family   
3      Fall 2010                 6/23/2015         Back Yard    Single-family   
4      Fall 2010                 6/23/2015         Back Yard    Single-family   

  DBH_baseline DB

In [8]:
# view columns in baseline and F23 to see how coordinate data is organized
print(baseline.head())

  OID_     ID PLANTED       SEASON FORESTER    STREET PROPERTY      TOWN  \
0    1  13377     Yes  Spring 2011       AP  ABBEY RD       18  Boylston   
1    2  13376     Yes  Spring 2011       AP  ABBEY RD       18  Boylston   
2    3  13378     Yes  Spring 2011       AP  ABBEY RD       18  Boylston   
3    4  13393     Yes  Spring 2011       AP  ABBEY RD       18  Boylston   
4    5  13396     Yes  Spring 2011       AP  ABBEY RD       18  Boylston   

             SPECIES                   EMAIL PARTIC                  NAME  \
0  Littleleaf Linden  lizfitz316@hotmail.com    Yes  Elizabeth Fitchgerld   
1  Littleleaf Linden  lizfitz316@hotmail.com    Yes  Elizabeth Fitchgerld   
2  Littleleaf Linden  lizfitz316@hotmail.com    Yes  Elizabeth Fitchgerld   
3  Littleleaf Linden  lizfitz316@hotmail.com    Yes  Elizabeth Fitchgerld   
4              Beech  lizfitz316@hotmail.com    Yes  Elizabeth Fitchgerld   

          PHONE LOCATION CALIPER IRRIGATION REPLACEMEN  \
0  508-869-6943  Priva

In [9]:
print(F23.head())

  OBJECTID    ID              SPECIES     STREET HOUSE_NUMBER       TOWN  \
0      187  1004        Kousa Dogwood  ARARAT ST          111  Worcester   
1      190  1015  Japanese Tree Lilac  ARARAT ST          171  Worcester   
2      191  1016        Kousa Dogwood  ARARAT ST          171  Worcester   
3      184  1017        Kousa Dogwood  ARARAT ST          171  Worcester   
4      186  1018       Sargent Cherry  ARARAT ST          171  Worcester   

  SEASON_PLANTED Observation_date_baseline SiteType_baseline LandUse_baseline  \
0      Fall 2010            6/24/2014 0:00                BY            SFR-D   
1      Fall 2010             7/7/2015 0:00                FY            SFR-D   
2      Fall 2010             7/7/2015 0:00                FY            SFR-D   
3      Fall 2010             7/7/2015 0:00                BY            SFR-D   
4      Fall 2010             7/7/2015 0:00                BY            SFR-D   

  DBH_baseline DBH_Height_baseline DBH2_baseline DBH2_He

In [10]:
# LAT and LON are Mass State Plane coordinates. 
# POINT_X and POINT_Y are EPSG 4326

# F23 already has the EPSG coordinates, but we should rename the columns to match as well while we are at it
F23 = F23.rename(columns={'x': 'POINT_X'})
F23 = F23.rename(columns={'y': 'POINT_Y'})

In [11]:
# Bring ESPSG coordinates to S23 as well
S23 = pd.merge(S23, baseline[['ID', 'POINT_X', 'POINT_Y']], on='ID', how='left')

In [12]:
print(S23.head())

  OBJECTID   ID OID_          SPECIES        STREET HOUSE_NUMBER       TOWN  \
0        1  956  110   Sargent Cherry  ACUSHNET AVE            9  Worcester   
1        2  962   34     Dawn Redwood     AIRLIE ST           70  Worcester   
2        3  963   48     Serviceberry     AIRLIE ST           70  Worcester   
3        4  964   35  Swamp White Oak     AIRLIE ST           70  Worcester   
4        5  965   38         Blackgum     AIRLIE ST           74  Worcester   

  SEASON_PLANTED Observation_Date_baseline SiteType_baseline LandUse_baseline  \
0      Fall 2010                        NA                NA               NA   
1      Fall 2010                 6/23/2015         Back Yard    Single-family   
2      Fall 2010                 6/23/2015        Front yard    Single-family   
3      Fall 2010                 6/23/2015         Back Yard    Single-family   
4      Fall 2010                 6/23/2015         Back Yard    Single-family   

  DBH_baseline DBH_Height_baseline DBH

In [13]:
# Now it's time to download these as .csv files and finalize column coordination
# Save DataFrame S23 as a CSV file
S23.to_csv('S23_DataPrep.csv', index=False)

# Save DataFrame F23 as a CSV file
F23.to_csv('F23_DataPrep.csv', index=False)